In [4]:
!pip install tavily-python

  Using cached tiktoken-0.8.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached regex-2024.11.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
Using cached tiktoken-0.8.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
Using cached regex-2024.11.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (796 kB)


In [9]:
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env", override=True)

True

In [18]:
{"role": "user", "content": f"다음 정보를 참고하여 '{term}'에 대한 설명을 작성하세요:\n\n{search_data}\n\n"
                                     f"오직 사진 촬영과 카메라 기술과 관련된 정보만 사용하여 용어를 설명하세요. "
                                     f"카메라와 무관한 의미(예: 국제 표준화 기구)는 포함하지 마세요."}

{'role': 'user',
 'content': '다음 정보를 참고하여 \'화이트 벨런스\'에 대한 설명을 작성하세요:\n\n화이트 밸런스(White Balance)는 사진이나 영상에서 색상을 정확하게 표현하기 위해 조정하는 과정이에요. 이 기능은 특히 조명이 다양한 환경에서 촬영할 때 중요하죠. 화이트 밸런스를 잘 맞추면, 촬영한 이미지나 영상에서 색상이 자연스럽고 현실적으로 보이게\n디지털 사진의 화이트 밸런스 이해. 화이트 밸런스는 색상을 보다 사실적으로 나타내기 위한 디지털 사진의 조정 기능입니다. "흰색을 흰색으로 보이도록 만들기 위해 사진을 중성색으로 설정하는 방법입니다"라고 사진작가이자 교육자인 Adam Long은 말합니다.\n화이트밸런스 기능은 이 색온도를 카메라가 감지해서 알맞은 값으로 조정하는 것이라고 할 수 있겠습니다. 화이트밸런스는 말 그대로 흰색을 기준으로 다른 색들도 본래의 색으로 찍히도록 하는 기능이며, 역으로 본래가 아닌 색으로 찍히도록 활용해서 개성\n\n오직 사진 촬영과 카메라 기술과 관련된 정보만 사용하여 용어를 설명하세요. 카메라와 무관한 의미(예: 국제 표준화 기구)는 포함하지 마세요.'}

In [10]:
from tavily import TavilyClient
import os

tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

In [ ]:
# Step 5. 사용자 질문 입력 및 응답 생성
query = "카메라에서 조리개 값이 사진에 미치는 영향은?"  # 예제 질문
response = qa_chain.run(query)
print("AI 응답:", response)

# Step 6. 전문 용어 추출 (정규 표현식 + GPT 활용 가능)
def extract_terms(text):
    camera_terms = re.findall(r"\b(ISO|조리개|셔터 스피드|화이트 밸런스|초점 거리|노출 보정|RAW|JPEG|EVF|광학 줌)\b", text, re.IGNORECASE)
    return list(set(camera_terms))  # 중복 제거

# 추출된 용어 확인
extracted_terms = extract_terms(response)
print("추출된 용어:", extracted_terms)


In [ ]:
from langchain.adapters.openai import convert_openai_messages
from langchain_community.chat_models import ChatOpenAI
from tavily import TavilyClient

In [17]:
# Step 7. Tavily를 이용한 용어별 웹 검색 실행
def search_term(term):
    search_results = tavily_client.search(term, search_depth="advanced")["results"]
    return "\n".join([result["content"] for result in search_results[:3]])  # 상위 3개 결과를 반환

# Step 8. GPT-4o를 사용한 용어 설명 생성
def explain_term(term, search_data):
    prompt = [
        {"role": "system", "content": "당신은 카메라 전문 용어를 쉽게 설명하는 전문가입니다."},
        {"role": "user", "content": f"다음 정보를 참고하여 '{term}'에 대한 설명을 작성하세요:\n\n{search_data}"}
    ]
    
    lc_messages = convert_openai_messages(prompt)
    explanation = ChatOpenAI(model="gpt-4o", openai_api_key=os.getenv("OPENAI_API_KEY")).invoke(lc_messages).content
    return explanation
extracted_terms = ["화이트 벨런스"]
# Step 9. 각 용어에 대한 설명 생성
term_explanations = {}
for term in extracted_terms:
    search_data = search_term(term)
    explanation = explain_term(term, search_data)
    term_explanations[term] = explanation

# 결과 출력
for term, explanation in term_explanations.items():
    print(f"🔍 용어: {term}\n설명: {explanation}\n")


🔍 용어: 화이트 벨런스
설명: 화이트 밸런스는 사진이나 영상에서 색상을 자연스럽고 정확하게 표현하기 위해 사용하는 카메라의 중요한 기능입니다. 이는 특히 조명이 다양한 환경에서 촬영할 때 더욱 중요해요. 예를 들어, 실내 조명은 종종 노란색이나 주황색 빛을 띠고, 야외의 햇빛은 푸르스름할 수 있습니다. 이런 상황에서 화이트 밸런스를 제대로 맞추지 않으면 사진이나 영상의 색상이 왜곡되어 보일 수 있죠.

화이트 밸런스는 카메라가 다양한 조명 환경에서 색온도를 감지하여 흰색이 본래의 흰색으로 보이도록 조정하는 과정을 말합니다. 이 과정을 통해 다른 색상도 자연스럽고 현실적인 색으로 표현될 수 있습니다. 사진작가 Adam Long은 이를 "흰색을 흰색으로 보이도록 만드는 방법"이라고 설명합니다.

또한, 화이트 밸런스를 창의적으로 사용하여 특정 분위기나 느낌을 표현할 수도 있습니다. 예를 들어, 따뜻한 느낌을 주고 싶다면 화이트 밸런스를 조정하여 이미지에 노란색 톤을 더할 수 있습니다. 반대로, 차가운 느낌을 원한다면 푸른색 톤을 강조할 수도 있죠. 따라서, 화이트 밸런스는 단순히 색을 정확히 표현하는 역할을 넘어, 창의적인 연출에도 중요한 도구가 됩니다.



In [ ]:
from pydantic import BaseModel, Field

class InputState(BaseModel):
    brand: str
    model: str
    question: str
    sessionId: int = Field(default=None, description="없으면 새로운 session 생성")
    